In [198]:
from __future__ import division
from __future__ import print_function
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import numpy as np
import pandas as pd
import scipy.sparse as sp
import time
import argparse
import numpy as np
import easydict
import math
import networkx as nx
# from .layers import *
# from .models import *
# from .utils import *
from torch.nn.parameter import Parameter
from torch.nn.modules.module import Module
from setuptools import setup
from setuptools import find_packages
# from pygcn.utils import load_data, accuracy
# from pygcn.models import GCN
# from pygcn.layers import GraphConvolution

import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
from torchvision import models
from skimage.io import imread
from skimage.transform import resize

In [199]:
df = pd.read_excel('/workspace/src/IgAN/brca-clinicalforwiki.xls')
df

,bcr_patient_barcode,age_at_initial_pathologic_diagnosis,ajcc_cancer_metastasis_stage_code,ajcc_neoplasm_disease_lymph_node_stage,ajcc_neoplasm_disease_stage,ajcc_tumor_stage_code,anatomic_organ_subdivision,axillary_lymph_node_stage_method_type,breast_cancer_optical_measurement_histologic_type,breast_cancer_surgery_margin_status,...,lymph_node_examined_count,margin_status,menopause_status,number_of_lymphnodes_positive_by_he,patient_id,pretreatment_history,prior_diagnosis,race,tissue_source_site,vital_status
0,TCGA-E2-A107,54,M0,N2a,Stage IIIA,T3,Right Upper Inner Quadrant,Sentinel lymph node biopsy plus axillary disse...,Infiltrating Lobular,[Not Available],...,27,Negative,Post (prior bilateral ovariectomy OR >12 mo si...,4,A107,NO,NO,WHITE,E2,LIVING
1,TCGA-E2-A108,64,M0,N2a,Stage IIIA,T2,Right Upper Outer Quadrant,[Not Available],Infiltrating Ductal,[Not Available],...,27,Negative,Post (prior bilateral ovariectomy OR >12 mo si...,4,A108,NO,NO,WHITE,E2,LIVING
2,TCGA-E2-A109,64,M0,N0 (i-),Stage IIA,T2,Right Upper Outer Quadrant,Sentinel node biopsy alone,Infiltrating Ductal,[Not Available],...,6,Negative,Post (prior bilateral ovariectomy OR >12 mo si...,0,A109,NO,NO,WHITE,E2,LIVING
3,TCGA-E2-A10B,67,M0,N1a,Stage IIB,T2,Left Lower Outer Quadrant,Sentinel lymph node biopsy plus axillary disse...,Infiltrating Ductal,[Not Available],...,28,Close,Post (prior bilateral ovariectomy OR >12 mo si...,2,A10B,NO,NO,WHITE,E2,LIVING
4,TCGA-E2-A10C,54,M0,N1a,Stage IIB,T2,Left Upper Outer Quadrant,Sentinel lymph node biopsy plus axillary disse...,Infiltrating Ductal,[Not Available],...,13,Negative,Post (prior bilateral ovariectomy OR >12 mo si...,1,A10C,NO,NO,WHITE,E2,LIVING
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
329,TCGA-AO-A0JF,68,M0,N1a,Stage IIA,T1c,Left Upper Outer Quadrant|Left,Sentinel lymph node biopsy plus axillary disse...,Infiltrating Ductal,[Not Available],...,14,Negative,Post (prior bilateral ovariectomy OR >12 mo si...,2,A0JF,NO,NO,WHITE,AO,LIVING
330,TCGA-AO-A0JJ,54,M0,N1a,Stage IIB,T2,Left Upper Outer Quadrant|Left,Sentinel lymph node biopsy plus axillary disse...,Infiltrating Lobular,[Not Available],...,44,Negative,Post (prior bilateral ovariectomy OR >12 mo si...,2,A0JJ,NO,NO,WHITE,AO,LIVING
331,TCGA-AO-A0JL,59,M0,N2a,Stage IIIA,T2,Left Upper Outer Quadrant|Left Lower Outer Qua...,Axillary lymph node dissection alone,Infiltrating Ductal,[Not Available],...,26,Negative,Post (prior bilateral ovariectomy OR >12 mo si...,7,A0JL,NO,NO,WHITE,AO,LIVING
332,TCGA-AO-A0JM,40,M0,N1,Stage IIB,T2,Right Lower Inner Quadrant|Right Lower Outer Q...,Sentinel lymph node biopsy plus axillary disse...,Infiltrating Ductal,[Not Available],...,23,Negative,Pre (<6 months since LMP AND no prior bilatera...,2,A0JM,NO,NO,[Not Available],AO,LIVING


In [200]:
def load_data(path="/workspace/src/IgAN/gcn/brca/", dataset="brca"):
    """Load citation network dataset (cora only for now)"""
    print('Loading {} dataset...'.format(dataset))
    
    #데이터의 구성: Node, Features, Label, Edge
    idx_features_labels = np.genfromtxt("{}{}.content".format(path, dataset),
                                        dtype=np.dtype(str))
    features = sp.csr_matrix(idx_features_labels[:, 1:-1], dtype=np.float32)
    labels = encode_onehot(idx_features_labels[:, -1])

    # build graph
    #각 노드의 index 부여
    idx = np.array(idx_features_labels[:, 0], dtype=np.int32)
    idx_map = {j: i for i, j in enumerate(idx)}
    #각 노드 간 연결 edge 데이터를 load하고 index로 다시 부여
    edges_unordered = np.genfromtxt("{}{}.cites".format(path, dataset),
                                    dtype=np.int32)
    edges = np.array(list(map(idx_map.get, edges_unordered.flatten())),
                     dtype=np.int32).reshape(edges_unordered.shape)
    adj = sp.coo_matrix((np.ones(edges.shape[0]), (edges[:, 0], edges[:, 1])),
                        shape=(labels.shape[0], labels.shape[0]),
                        dtype=np.float32)

    # build symmetric adjacency matrix
    adj = adj + adj.T.multiply(adj.T > adj) - adj.multiply(adj.T > adj)

    features = normalize(features)
    adj = normalize(adj + sp.eye(adj.shape[0]))
    #train, val, test에 index 지정하여 semi-supervised transductive를 사용할 수 있도록
    idx_train = range(134, 334)
    idx_val = range(67, 134)
    idx_test = range(0, 67)

    features = torch.FloatTensor(np.array(features.todense()))
    labels = torch.LongTensor(np.where(labels)[1])
    adj = sparse_mx_to_torch_sparse_tensor(adj)

    idx_train = torch.LongTensor(idx_train)
    idx_val = torch.LongTensor(idx_val)
    idx_test = torch.LongTensor(idx_test)
    return adj, features, labels, idx_train, idx_val, idx_test

In [201]:
#class의 label을 onehot으로 변환
def encode_onehot(labels):
    classes = set(labels)
    classes_dict = {c: np.identity(len(classes))[i, :] for i, c in
                    enumerate(classes)}
    labels_onehot = np.array(list(map(classes_dict.get, labels)),
                             dtype=np.int32)
    return labels_onehot

In [202]:
#row-wise로 feature, edge weights를 normalize
def normalize(mx):
    """Row-normalize sparse matrix"""
    rowsum = np.array(mx.sum(1))
    r_inv = np.power(rowsum, -1).flatten()
    r_inv[np.isinf(r_inv)] = 0.
    r_mat_inv = sp.diags(r_inv)
    mx = r_mat_inv.dot(mx)
    return mx

In [203]:
def accuracy(output, labels):
    preds = output.max(1)[1].type_as(labels)
    correct = preds.eq(labels).double()
    correct = correct.sum()
    return correct / len(labels)

In [204]:
def sparse_mx_to_torch_sparse_tensor(sparse_mx):
    """Convert a scipy sparse matrix to a torch sparse tensor."""
    sparse_mx = sparse_mx.tocoo().astype(np.float32)
    indices = torch.from_numpy(np.vstack((sparse_mx.row, sparse_mx.col)).astype(np.int64))
    values = torch.from_numpy(sparse_mx.data)
    shape = torch.Size(sparse_mx.shape)
    return torch.sparse.FloatTensor(indices, values, shape)

In [205]:
class GraphConvolution(Module):
    """
    Simple GCN layer, similar to https://arxiv.org/abs/1609.02907
    """

    def __init__(self, in_features, out_features, bias=True):
        super(GraphConvolution, self).__init__()
        self.in_features = in_features
        self.out_features = out_features
        #parameter W 정의
        self.weight = Parameter(torch.FloatTensor(in_features, out_features))
        if bias:
            self.bias = Parameter(torch.FloatTensor(out_features))
        else:
            self.register_parameter('bias', None)
        self.reset_parameters()

    def reset_parameters(self):
        stdv = 1. / math.sqrt(self.weight.size(1))
        self.weight.data.uniform_(-stdv, stdv)
        if self.bias is not None:
            self.bias.data.uniform_(-stdv, stdv)

    def forward(self, input, adj):
        #input feature와 W 계산
        support = torch.mm(input, self.weight)
        #A와 XW 계산
        output = torch.spmm(adj, support)
        if self.bias is not None:
            return output + self.bias
        else:
            return output

    def __repr__(self):
        return self.__class__.__name__ + ' (' \
               + str(self.in_features) + ' -> ' \
               + str(self.out_features) + ')'

In [229]:
class GCN(nn.Module):
    def __init__(self, nfeat, nhid, nclass, dropout):
        super(GCN, self).__init__()
        #정의된 layer를 불러와 GCN 연산
        self.gc1 = GraphConvolution(nfeat, nhid)
        self.gc2 = GraphConvolution(nhid, nclass)
        self.dropout = dropout

    def forward(self, x, adj):
        global l0_features 
        global l1_features 
        global l2_features
        if epoch == 99:
            l0_features = x
            print("Before")
            print("Node Feature Matrix", l0_features)
        x = F.relu(self.gc1(x, adj))
        x = F.dropout(x, self.dropout, training=self.training)
        if epoch == 99:
            l1_features = x
            print("First Layer")
            print("Node Feature Matrix", l1_features)
        x = self.gc2(x, adj)
        if epoch == 99:
            l2_features = x
            print("Second Layer")
            print("Node Feature Matrix", l2_features)
        #log_softmax를 취함
        return F.log_softmax(x, dim=1)

In [230]:
# Training settings
args = easydict.EasyDict({
    "no-cuda": False,
    "fastmode": False,
    "seed": 42,
    "epoch": 100,
    "lr": 0.01,
    "weight_decay": 5e-4,
    "hidden": 16,
    "dropout": 0.5
})
np.random.seed(args.seed)
torch.manual_seed(args.seed)
torch.cuda.manual_seed(args.seed)

In [231]:
# Load data
adj, features, labels, idx_train, idx_val, idx_test = load_data()

# Model and optimizer
model = GCN(nfeat=features.shape[1],
            nhid=args.hidden,
            nclass=labels.max().item() + 1,
            dropout=args.dropout)
optimizer = optim.Adam(model.parameters(), lr=args.lr, weight_decay=args.weight_decay)

Loading brca dataset...


In [232]:
print(l0_features)

tensor([[0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0500, 0.0000],
        [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0500, 0.0000],
        [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0500, 0.0000],
        ...,
        [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0500, 0.0000],
        [0.0000, 0.0000, 0.0500,  ..., 0.0000, 0.0000, 0.0500],
        [0.0000, 0.0000, 0.0500,  ..., 0.0000, 0.0500, 0.0000]])


In [235]:
t_np = l1_features.detach().numpy() #convert to Numpy array
df = pd.DataFrame(t_np) #convert to a dataframe
df.to_csv("testfile.csv",index=False) #save to file

In [217]:
model

GCN(
  (gc1): GraphConvolution (198 -> 16)
  (gc2): GraphConvolution (16 -> 2)
)

In [218]:
print('adj :', adj.shape)
print('features :', features.shape)
print('labels :', labels.shape)
print('idx :', idx_train.shape, idx_val.shape, idx_test.shape)

adj : torch.Size([334, 334])
features : torch.Size([334, 198])
labels : torch.Size([334])
idx : torch.Size([200]) torch.Size([67]) torch.Size([67])


In [219]:
def train(epoch):
    t = time.time()
    model.train()
    optimizer.zero_grad()
    output = model(features, adj)
    
    #앞에서 log_softmax를 취했기 떄문에 nll_loss 사용
    #semi-supervised 접근을 위해 train의 index만 loss 계산하고 측정
    loss_train = F.nll_loss(output[idx_train], labels[idx_train])
    acc_train = accuracy(output[idx_train], labels[idx_train])
    loss_train.backward()
    optimizer.step()

    if not args.fastmode:
        # Evaluate validation set performance separately,
        # deactivates dropout during validation run.
        model.eval()
        output = model(features, adj)
    
    #valid loss 계산
    loss_val = F.nll_loss(output[idx_val], labels[idx_val])
    acc_val = accuracy(output[idx_val], labels[idx_val])
    if (((epoch+1) % 10) == 0) :
        print('Epoch: {:04d}'.format(epoch+1),
              'loss_train: {:.4f}'.format(loss_train.item()),
              'acc_train: {:.4f}'.format(acc_train.item()),
              'loss_val: {:.4f}'.format(loss_val.item()),
              'acc_val: {:.4f}'.format(acc_val.item()),
              'time: {:.4f}s'.format(time.time() - t))

In [220]:
def test():
    #test loss 및 accuracy 계산
    model.eval()
    output = model(features, adj)
    loss_test = F.nll_loss(output[idx_test], labels[idx_test])
    acc_test = accuracy(output[idx_test], labels[idx_test])
    print("Test set results:",
          "loss= {:.4f}".format(loss_test.item()),
          "accuracy= {:.4f}".format(acc_test.item()))

In [221]:
# Train model
t_total = time.time()
for epoch in range(args.epoch):
    train(epoch)
print("Optimization Finished!")
print("Total time elapsed: {:.4f}s".format(time.time() - t_total))

Epoch: 0010 loss_train: 0.5107 acc_train: 0.7850 loss_val: 0.2263 acc_val: 0.9701 time: 0.0060s
Epoch: 0020 loss_train: 0.5060 acc_train: 0.7850 loss_val: 0.2651 acc_val: 0.9701 time: 0.0056s
Epoch: 0030 loss_train: 0.4912 acc_train: 0.7850 loss_val: 0.2839 acc_val: 0.9701 time: 0.0056s
Epoch: 0040 loss_train: 0.4764 acc_train: 0.7850 loss_val: 0.2436 acc_val: 0.9701 time: 0.0057s
Epoch: 0050 loss_train: 0.4492 acc_train: 0.7900 loss_val: 0.2396 acc_val: 0.9701 time: 0.0056s
Epoch: 0060 loss_train: 0.4233 acc_train: 0.8000 loss_val: 0.2317 acc_val: 0.9701 time: 0.0056s
Epoch: 0070 loss_train: 0.3910 acc_train: 0.8350 loss_val: 0.2058 acc_val: 0.9701 time: 0.0045s
Epoch: 0080 loss_train: 0.3509 acc_train: 0.8600 loss_val: 0.1921 acc_val: 0.9701 time: 0.0043s
Epoch: 0090 loss_train: 0.3263 acc_train: 0.8600 loss_val: 0.1742 acc_val: 0.9701 time: 0.0047s
Before
Node Feature Matrix tensor([[0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0500, 0.0000],
        [0.0000, 0.0000, 0.0000,  ..., 0.0000

In [196]:
# Testing
test()

Before
Node Feature Matrix torch.Size([334, 198])
First Layer
Node Feature Matrix torch.Size([334, 16])
Second Layer
Node Feature Matrix torch.Size([334, 2])
Test set results: loss= 0.3561 accuracy= 0.8806
